# Device & Patch Extractors


In [1]:
%load_ext autoreload
%autoreload all
%env LOGGING_LEVEL=WARNING

env: LOGGING_LEVEL=WARNING


In [2]:
import typing

from termcolor import colored

from hbit import bootstrap, core, prompting, types
from hbit.extractors import device_extractors, patch_extractors

/Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
service_factory = bootstrap.ServicesFactory()
service_factory.add_db()
service_factory.add_requests()
service_factory.add_client()
service_factory.add_chat_prompt_templates()
service_factory.add_openai_models(requests_per_second=1)

registry = service_factory.registry
registry

/Users/macbook/Projects/hbit/hbit/bootstrap.py:117: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [4]:
db = registry.get_service(core.DatabaseService)
prompting_store = registry.get_service(prompting.PromptStore)
model = registry.get_service(types.ExtractionModel)
model

ChatOpenAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x1210893a0>, client=<openai.resources.chat.completions.Completions object at 0x1210c4740>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1210c6510>, root_client=<openai.OpenAI object at 0x1210aea20>, root_async_client=<openai.AsyncOpenAI object at 0x1210c47a0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), seed=0)

In [5]:
class QuestionAnswer(typing.NamedTuple):
    text: str
    patch_build: str | None
    device_identifier: str | None


texts = [
    QuestionAnswer(
        text="Write a summary of how secure is an iPhone XS running iOS 17.7.2? Are there any known vulnerabilities or security concerns with this version?",
        patch_build="21h221",
        device_identifier="iphone11,2",
    ),
    QuestionAnswer(
        text="How security is my iPhone 13 Pro device if I have patch 18.1.0 installed identified by build 22B83.",
        patch_build="22b83",
        device_identifier="iphone14,2",
    ),
    QuestionAnswer(
        text="What version does my patch with build 22B83 have?",
        patch_build="22b83",
        device_identifier=None,
    ),
    QuestionAnswer(
        text="Can you explain if iPhone 12 running iOS 16.5.1 is safe from known vulnerabilities?",
        patch_build="20f75",
        device_identifier="iphone13,2",
    ),
    QuestionAnswer(
        text="What security improvements are introduced in iOS 17.0.3 for the iPhone 14 Pro Max?",
        patch_build="21a360",
        device_identifier="iphone15,3",
    ),
    QuestionAnswer(
        text="Is my iPhone SE (2nd Gen) secure if it's updated to iOS 15.7.4 with build number 19H321?",
        patch_build="19h321",
        device_identifier=None,
    ),
    QuestionAnswer(
        text="Are there any critical security concerns with iOS 16.6.0 on the iPhone 11 Pro?",
        patch_build="20g75",
        device_identifier="iphone12,3",
    ),
    QuestionAnswer(
        text="What known bugs or vulnerabilities exist for iPhone XR devices running iOS 15.2.1?",
        patch_build="19c63",
        device_identifier="iphone11,8",
    ),
    QuestionAnswer(
        text="How does the iOS 17.1.1 patch improve security on iPhone 13 devices? What is its build number?",
        patch_build="21b91",
        device_identifier="iphone14,5",
    ),
    QuestionAnswer(
        text="Can you confirm if my iPhone XS Max running iOS 15.2 build 19C56 is fully secure?",
        patch_build="19c56",
        device_identifier="iphone11,6",
    ),
    QuestionAnswer(
        text="What's the latest iOS patch for iPhone 14, and does it address critical security vulnerabilities?",
        patch_build=None,
        device_identifier="iphone14,7",
    ),
    QuestionAnswer(
        text="Are there any concerns with iPhone 12 Mini running iOS 17.6.2 build 21G123?",
        patch_build=None,
        device_identifier="iphone13,1",
    ),
    QuestionAnswer(
        text="What issues might arise from using an iPhone 11?",
        patch_build=None,
        device_identifier="iphone12,1",
    ),
]

In [6]:
index = 8
text = texts[index].text
text

'How does the iOS 17.1.1 patch improve security on iPhone 13 devices? What is its build number?'

## SQL Device Extractor

In [7]:
sql_device_extractor = device_extractors.SqlDeviceExtractor(
    model=model, db=db, prompt_store=prompting_store
)
sql_device_extractor

In [8]:
sql_device_extractor.extract_device_identifier(text)

## Structured Device Extractor


In [9]:
structured_device_extractor = device_extractors.JsonDeviceExtractor(
    model=model, db=db, prompt_store=prompting_store
)
structured_device_extractor

In [10]:
structured_device_extractor.extract_device_identifier(text)

'iphone14,5'

## SQL Patch Extractor


In [11]:
sql_patch_extractor = patch_extractors.SqlPatchExtractor(
    model=model, db=db, prompt_store=prompting_store
)
sql_patch_extractor

In [12]:
sql_patch_extractor.extract_patch_build(text)

'21b91'

## Structured Patch Extractor


In [13]:
structured_patch_extractor = patch_extractors.JsonPatchExtractor(
    model=model, db=db, prompt_store=prompting_store
)
structured_patch_extractor

In [14]:
structured_patch_extractor.extract_patch_build(text)

'21b91'

## Test all And Answers

In [15]:
for i, (text, patch_build, device_identifier) in enumerate(texts):
    print(
        f"================================== Example {i} =================================="
    )
    print(f"Text: {text}")

    structured_device_identifier = (
        structured_device_extractor.extract_device_identifier(text)
    )
    structured_color = (
        "green" if structured_device_identifier == device_identifier else "red"
    )
    sql_device_identifier = sql_device_extractor.extract_device_identifier(text)
    sql_color = "green" if sql_device_identifier == device_identifier else "red"

    print(
        f"Device Extractor: Expected: {colored(device_identifier, 'green')}, "
        f"Structured Response: {colored(structured_device_identifier, structured_color)}, "
        f"SQL Response: {colored(sql_device_identifier, sql_color)}"
    )

    structured_patch_build = structured_patch_extractor.extract_patch_build(text)
    structured_color = "green" if structured_patch_build == patch_build else "red"
    sql_patch_build = sql_patch_extractor.extract_patch_build(text)
    sql_color = "green" if sql_patch_build == patch_build else "red"

    print(
        f"Patch Extractor: Expected: {colored(patch_build, 'green')}, "
        f"Structured Response: {colored(structured_patch_build, structured_color)}, "
        f"SQL Response: {colored(sql_patch_build, sql_color)}"
    )
    print()

================================== Example 0 ==================================
Text: Write a summary of how secure is an iPhone XS running iOS 17.7.2? Are there any known vulnerabilities or security concerns with this version?
Device Extractor: Expected: iphone11,2, Structured Response: iphone11,2, SQL Response: None
Patch Extractor: Expected: 21h221, Structured Response: 21h221, SQL Response: None

================================== Example 1 ==================================
Text: How security is my iPhone 13 Pro device if I have patch 18.1.0 installed identified by build 22B83.
Device Extractor: Expected: iphone14,2, Structured Response: iphone14,2, SQL Response: None
Patch Extractor: Expected: 22b83, Structured Response: 22b83, SQL Response: 22b83

================================== Example 2 ==================================
Text: What version does my patch with build 22B83 have?
Device Extractor: Expected: None, Structured Response: None, SQL Response: None
Patch Extractor: Ex